# Risk Parity

# Install packages missing

In [ ]:
pip install wrds

# Import packages

In [ ]:
import os
os.getcwd()
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2
import wrds
from random import *
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import math
import matplotlib.pyplot as plt
import requests
import pandas_datareader
from datetime import timedelta
from scipy.stats import norm
import scipy.optimize as optimize
import sympy as sym
import seaborn as sns
import datetime as dt
import yfinance as yf
from datetime import datetime

from pandas.tseries.offsets import *

# Settings: shares code, exchange codes to use, sample period, folder for pictures/temporary data/outout
use_NYSE_brkpts = False

In [ ]:
import os
os.getcwd()
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2
import wrds
from random import *
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import math
import matplotlib.pyplot as plt
import requests
import pandas_datareader
from datetime import timedelta
from scipy.stats import norm
import scipy.optimize as optimize
import sympy as sym
import seaborn as sns
import datetime as dt
import yfinance as yf
from datetime import datetime

from pandas.tseries.offsets import *

# Settings: shares code, exchange codes to use, sample period, folder for pictures/temporary data/outout
use_NYSE_brkpts = False

# API Connection

In [ ]:
# API Connection
load_data = Trues
wrds_id = 'username'
conn = wrds.Connection(wrds_username=wrds_id)
#conn.create_pqpass_file()
conn.close()

conn = wrds.Connection(wrds_username=wrds_id)
# Load CRSP returns and change variables format
crsp_bonds_raw = conn.raw_sql("""
                        select kycrspid, mcaldt, tmretnua, tmtotout
                        from crspq.tfz_mth
                        """)
conn.close()

#                        where a.date between '01/01/1926' and '12/21/2022'

crsp_bonds_raw['mcaldt'] = pd.to_datetime(crsp_bonds_raw['mcaldt'],format='%Y-%m-%d',errors='ignore')

Enter your WRDS username [mjgarciagomez]:mjgarciagomez
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Loading library list...
Done


<ipython-input-4-171d82016041>:18: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  crsp_bonds_raw['mcaldt'] = pd.to_datetime(crsp_bonds_raw['mcaldt'],format='%Y-%m-%d',errors='ignore')


# Importing Bonds information from WRDS

In [ ]:
# Connection Bonds
conn = wrds.Connection(wrds_username=wrds_id)
# Load CRSP returns and change variables format
crsp_bonds_raw = conn.raw_sql("""
                        select kycrspid, mcaldt, tmretnua, tmtotout
                        from crspq.tfz_mth
                        """)
conn.close()

#                        where a.date between '01/01/1926' and '12/21/2022'

crsp_bonds_raw['mcaldt'] = pd.to_datetime(crsp_bonds_raw['mcaldt'],format='%Y-%m-%d',errors='ignore')

Loading library list...
Done


<ipython-input-5-ca21108344a5>:12: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  crsp_bonds_raw['mcaldt'] = pd.to_datetime(crsp_bonds_raw['mcaldt'],format='%Y-%m-%d',errors='ignore')


# Saving Pickle

In [ ]:
#save pickle
crsp_bonds_raw.to_pickle('crsp_bonds.pkl')

In [ ]:
crsp_bonds = crsp_bonds_raw.copy()

crsp_bonds['Year']  = pd.DatetimeIndex(crsp_bonds.mcaldt).year
crsp_bonds['Month'] = pd.DatetimeIndex(crsp_bonds.mcaldt).month
crsp_bonds          = crsp_bonds.sort_values(by=['mcaldt'],ascending = True)

#crsp_bonds = crsp_bonds.dropna()

In [ ]:
crsp_bonds['me']    = crsp_bonds["tmtotout"]
crsp_bonds['lme']    = crsp_bonds.groupby('kycrspid')["me"].shift(1)

In [ ]:
ewret = crsp_bonds.groupby(['mcaldt'])['tmretnua'].mean()
ewret = ewret.to_frame()
ewret.columns=['Bond_Ew_Ret']

In [ ]:
vwret = crsp_bonds.groupby(['mcaldt'])[['lme','tmretnua']].apply(lambda x: np.sum(x['lme']/x['lme'].sum()*x['tmretnua'])) # Use a list to select multiple columns
vwret = vwret.to_frame()
vwret.columns=['Bond_Vw_Ret']

In [ ]:
total_me = crsp_bonds.groupby(['mcaldt'])['me'].sum().to_frame().rename(columns = {'me' : 'total_me'})
Bond_lag_MV = total_me.shift(1).rename(columns = {'total_me' : 'Bond_lag_MV'})

In [ ]:
outputQ1 = Bond_lag_MV
outputQ1 = pd.merge(outputQ1 , ewret,how = 'outer', left_index=True, right_index=True )
outputQ1 = pd.merge(outputQ1 , vwret,how = 'outer', left_index=True, right_index=True )
outputQ1 = outputQ1.reset_index()
outputQ1.insert (0, "Month", pd.DatetimeIndex(outputQ1.mcaldt).month)
outputQ1.insert (0, "Year", pd.DatetimeIndex(outputQ1.mcaldt).year)

In [ ]:
outputQ1_date = outputQ1.copy()
outputQ1_date = outputQ1_date.rename(columns = {'mcaldt': 'date'})
outputQ1_date['date_MY'] = outputQ1_date['date'].dt.to_period('M')

In [ ]:
outputQ1 = outputQ1.drop(columns = 'mcaldt')
outputQ1 = outputQ1.iloc[1:len(outputQ1),:] #drop 1925 december, which has NaN values
outputQ1_date = outputQ1_date.iloc[1:len(outputQ1_date),:]

      Year  Month  Bond_lag_MV  Bond_Ew_Ret  Bond_Vw_Ret
1     1926      1      19502.0     0.005101     0.006829
2     1926      2      19502.0     0.003621     0.003844
3     1926      3      19500.0     0.003812     0.003588
4     1926      4      19231.0     0.004014     0.006486
5     1926      5      19227.0     0.002146     0.002629
...    ...    ...          ...          ...          ...
1175  2023     11   25478472.0     0.032521     0.030095
1176  2023     12   25720925.0     0.027771     0.024899
1177  2024      1   25671379.0     0.000790     0.000688
1178  2024      2   25979221.0    -0.011430    -0.010299
1179  2024      3   26259737.0     0.005873     0.005391

[1179 rows x 5 columns]
Year             int32
Month            int32
Bond_lag_MV    float64
Bond_Ew_Ret    float64
Bond_Vw_Ret    float64
dtype: object


In [ ]:
print(outputQ1)
print(outputQ1.dtypes)

Loading library list...
Done


In [ ]:
# Load T-bill
ds_id = 'edgaradel'
conn = wrds.Connection(wrds_username=wrds_id)
#conn.create_pqpass_file()
conn.close()

Loading library list...
Done


In [ ]:
conn = wrds.Connection(wrds_username=wrds_id)

rf = conn.raw_sql("""
                      select caldt, t30ret, t90ret
                      from crspq.mcti
                      """)
rf['caldt']   = pd.DataFrame(rf[['caldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
rf = rf.rename(columns={"caldt":"date","t30ret":"rf30","t90ret":"rf90"}).copy()

conn.close()



rf['date_MY'] = rf['date'].dt.to_period('M')
rf = rf.iloc[1:len(rf),:]

Loading library list...
Done
Loading library list...
Done


<ipython-input-20-406717a1e95e>:24: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  crsp_raw['date'] = pd.to_datetime(crsp_raw['date'],format='%Y-%m-%d',errors='ignore')


In [ ]:
#repeat Steps PSet 1

#load data

wrds_id = 'mjgarciagomez'
conn = wrds.Connection(wrds_username=wrds_id)
#conn.create_pqpass_file()
conn.close()


conn = wrds.Connection(wrds_username=wrds_id)
# Load CRSP returns and change variables format
crsp_raw = conn.raw_sql("""
                        select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                        a.ret, a.shrout, a.prc
                        from crspq.msf as a
                        left join crsp.msenames as b
                        on a.permno=b.permno
                        and b.namedt<=a.date
                        and a.date<=b.nameendt
                        where a.date between '12/01/1925' and '12/31/2022'
                        """)
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'],format='%Y-%m-%d',errors='ignore')
#crsp_raw['date'] = pd.DataFrame(crsp_raw[['date']].values.astype('datetime64[ns]')) + MonthEnd(0)

# Load CRSP Delisting returns and change variables format
dlret_raw = conn.raw_sql("""
                        select permno, dlret, dlstdt, dlstcd
                        from crspq.msedelist
                        """)
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'], format='%Y-%m-%d')
dlret_raw['date']   = dlret_raw['dlstdt']+MonthEnd(0)
conn.close()

#save pickle
crsp_raw.to_pickle('crsp_raw.pkl')
dlret_raw.to_pickle('dlret_raw.pkl')

# Merging with delisting

In [ ]:
#cleaning PSet1

#Merging with delisting return (Fama and French)
crsp = pd.merge(crsp_raw, dlret_raw, how = 'left', on = ['permno', 'date'])

# Keeping common shares (share codes 10 and 11) and major exchanges (exchage code 1, 2 and 3)
crsp = crsp[(crsp['shrcd'] == 10)| (crsp['shrcd']  == 11)]
crsp = crsp[(crsp['exchcd']== 1) | (crsp['exchcd'] == 2) | (crsp['exchcd'] == 3)]

crsp = crsp.drop(['exchcd','shrcd'], axis = 1)
crsp = crsp.copy()

# Compute returns including delisting returns
aux = crsp[(crsp['ret'].isna() & crsp['dlret'].isna())].index
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret']   = crsp['ret'].fillna(0)
crsp['ret']   = (1 + crsp['ret'])*(1+crsp['dlret'])-1
crsp.loc[aux,['ret']] = np.nan
crsp = crsp.drop(['dlstcd'], axis=1)

# Computing Market Equity

In [ ]:
# Market Equity ( market capitalization)

crsp['me'] = crsp['prc'].abs()*crsp['shrout']  # calculate market equity
crsp       = crsp.drop(['dlret', 'dlstdt', 'prc', 'permco', 'shrout'], axis=1)

# Lagged Market Equity (to be used as weights)
crsp       = crsp.sort_values(by = ['permno', 'date']).reset_index().drop('index', axis=1).copy()
crsp['daten'] = crsp['date'].dt.year*12 + crsp['date'].dt.month
crsp['IsValidLag'] = crsp['daten'].diff(1) == 1 # Lag date has to be the lagged date
crsp.loc[crsp[crsp['permno'].diff(1) != 0].index, ['IsValidLag']] = False # Lagged data has to be the same security
crsp['Lme'] = crsp[['permno', 'me']].groupby('permno').shift(1)
crsp.loc[crsp[crsp['IsValidLag'] == False].index, ['Lme']] = np.nan
crsp = crsp.drop(['IsValidLag', 'daten'], axis=1)

# Substituting missing

In [ ]:
# Substituting missing returns with zero (no trade assumption)
# crsp.loc[crsp['ret'].isna(), 'ret'] = 0
# or drop missing returns altogether -- better fit to French's returns series
crsp = crsp.loc[crsp['Lme'] > 0, :].copy()

In [ ]:
# Dropping securities with missing lagged market cap data (zero weight in the market portfolio)
crsp = crsp.sort_values(by = ['date', 'permno']).reset_index().drop('index', axis = 1).copy()

# Use a list to select multiple columns
vwret = crsp.groupby(['date'])[['Lme','ret']].apply(lambda x: np.sum(x['Lme']/x['Lme'].sum()*x['ret']))
vwret = vwret.to_frame()
vwret.columns=['vwret']

In [ ]:
# equal weighted returns
ewret = crsp.groupby(['date'])['ret'].mean()
ewret = ewret.to_frame()
ewret.columns=['ewret']

#make lagged market eq variable
Lme = crsp[['date','Lme']]
Lme_millions = Lme.copy()
Lme_millions.Lme = Lme_millions.Lme/1000000
# Aggregate Market Equity
mktcap = crsp.groupby(['date'])['me'].sum().to_frame().reset_index().rename(columns = {'me' : 'mktcap'})
# crsp = crsp.drop(['vwret', 'w'], axis = 1).copy()

# Merge replicated series with French's data
output = mktcap.copy()
output = pd.merge(output, vwret, how = 'outer', on = ['date'])
output = pd.merge(output, ewret, how = 'outer', on = ['date'])

In [ ]:
mktcap_lagged = Lme_millions.groupby(['date'])['Lme'].sum().to_frame().reset_index().rename(columns = {'Lme' : 'lagged_mktcap'})

mktcap_mil = pd.merge(mktcap,mktcap_lagged, how = 'outer', on = 'date')

output = pd.merge(output, mktcap_mil, how = 'outer', on = ['date'])

In [ ]:
#using month-year to merge tbill rates and equities
output['date_MY'] = output.date.dt.to_period('M')
output = pd.merge(output, rf, how = 'outer', on = ['date_MY'])
output = pd.merge(output, outputQ1_date, how = 'outer', on = ['date_MY'])

output['Stock_Excess_Vw_Ret'] = output['vwret'] - output['rf30']
output['Bond_Excess_Vw_Ret'] = output['Bond_Vw_Ret'] - output['rf30']

output = output.rename(columns = {'lagged_mktcap':'Stock_lag_MV'})

      Year  Month  Stock_lag_MV  Stock_Excess_Vw_Ret  Bond_lag_MV  \
0     1926      1     26.903953            -0.002704      19502.0   
1     1926      2     27.032250            -0.036744      19502.0   
2     1926      3     26.128629            -0.067653      19500.0   
3     1926      4     24.506932             0.033663      19231.0   
4     1926      5     25.178699             0.011629      19227.0   
...    ...    ...           ...                  ...          ...   
1174  2023     11           NaN                  NaN   25478472.0   
1175  2023     12           NaN                  NaN   25720925.0   
1176  2024      1           NaN                  NaN   25671379.0   
1177  2024      2           NaN                  NaN   25979221.0   
1178  2024      3           NaN                  NaN   26259737.0   

      Bond_Excess_Vw_Ret  
0               0.003878  
1               0.001076  
2               0.000810  
3               0.003414  
4               0.002287  
...      

Aggregate stock, bond, and riskless datatables. For each year-month, calculate the lagged
market value and excess value-weighted returns for both stocks and bonds. Your output should
be from January 1926 to December 2022, at a monthly frequency.

In [ ]:
outputQ2 = output[['Year','Month','Stock_lag_MV','Stock_Excess_Vw_Ret','Bond_lag_MV','Bond_Excess_Vw_Ret']]

print(outputQ2)

print(outputQ2.dtypes)

Calculate the monthly unlevered and levered risk-parity portfolio returns as defined by Asness, Frazzini, and Pedersen (2012).3 For the levered risk-parity portfolio, match the value-weighted portfolio’s ˆσ over the longest matched holding period of both. Your output should be from January 1926 to December 2022, at a monthly frequency.

In [ ]:
df3 = output.copy()

df3['w_bond_port']  = df3.Bond_lag_MV / (df3.Bond_lag_MV+df3.Stock_lag_MV)
df3['w_stock_port'] = df3.Stock_lag_MV / (df3.Bond_lag_MV+df3.Stock_lag_MV)

df3['Excess_Vw_Ret'] =  df3['vwret']*df3['w_bond_port'] + df3['Bond_Vw_Ret']*df3['w_stock_port'] - df3['rf30']



df3['Excess_60_40_Ret'] = df3['vwret']*0.6 + df3['Bond_Vw_Ret']*0.4 - df3['rf30']
df3['Excess_60_40'] = df3['Excess_60_40_Ret'].cumsum()



def inverse_sigma_hat(df):
    sigma = df.std()
    inv_sigma = 1/sigma
    return inv_sigma

#note that I have added a .shift(1) as they use t-1:37, 3 year month window
df3['Stock_inverse_sigma_hat'] = df3.Stock_Excess_Vw_Ret.rolling(window=36,center=False).apply(lambda x: inverse_sigma_hat(x)).shift(1)
df3['Bond_inverse_sigma_hat'] = df3.Bond_Excess_Vw_Ret.rolling(window=36,center=False).apply(lambda x: inverse_sigma_hat(x)).shift(1)

#unlevered k is 1/sum(sigma_inv all asset classes)
# k_t = 1/(sigma_inv_stock_t + sigma_inv_bond_t)

df3['Unlevered_k'] = df3[['Stock_inverse_sigma_hat','Bond_inverse_sigma_hat']].apply(lambda x: 1/np.sum(x), axis=1)

df3['w_stock_unlevered'] = df3['Unlevered_k']*df3['Stock_inverse_sigma_hat']
df3['w_bond_unlevered'] = df3['Unlevered_k']*df3['Bond_inverse_sigma_hat']

df3['Excess_Unlevered_RP_Ret'] = df3['vwret']*df3['w_stock_unlevered'] + df3['Bond_Vw_Ret']*df3['w_bond_unlevered'] - df3['rf30']
df3['Excess_Unlevered_RP'] = df3['Excess_Unlevered_RP_Ret'].cumsum()
#choose leverd k = 0.1


sigma_market = df3.Excess_Vw_Ret.std()

df3['levered_k']  = 0.05

df3['w_stock_levered']   = df3['levered_k']*df3['Stock_inverse_sigma_hat']
df3['w_bond_levered']    = df3['levered_k']*df3['Bond_inverse_sigma_hat']
df3['w_rf_levered']      = -1*(df3['w_stock_levered'] + df3['w_bond_levered'] -1)
df3['Excess_Levered_RP_Ret'] = df3['vwret']*df3['w_stock_levered'] + df3['Bond_Vw_Ret']*df3['w_bond_levered'] + df3['w_rf_levered']*df3['rf30'] - df3['rf30']
df3['Excess_Levered_RP'] = df3['Excess_Levered_RP_Ret'].cumsum()


def sigma_levered_k(k_lev):
#    df3['levered_k'] = k_lev

    df3['w_stock_levered']   = k_lev*df3['Stock_inverse_sigma_hat']
    df3['w_bond_levered']    = k_lev*df3['Bond_inverse_sigma_hat']
    df3['w_rf_levered']      = -1*(df3['w_stock_levered'] + df3['w_bond_levered'] -1)
    df3['Excess_Levered_RP_Ret'] = df3['vwret']*df3['w_stock_levered'] + df3['Bond_Vw_Ret']*df3['w_bond_levered'] + df3['w_rf_levered']*df3['rf30'] - df3['rf30']

    return np.abs(sigma_market-df3['Excess_Levered_RP_Ret'].std())

result = optimize.minimize(sigma_levered_k , 10 )


print(result)

df3['levered_k'] = result.x[0]
df3['w_stock_levered']   = df3['levered_k']*df3['Stock_inverse_sigma_hat']
df3['w_bond_levered']    = df3['levered_k']*df3['Bond_inverse_sigma_hat']
df3['w_rf_levered']      = -1*(df3['w_stock_levered'] + df3['w_bond_levered'] -1)
df3['Excess_Levered_RP_Ret'] = df3['vwret']*df3['w_stock_levered'] + df3['Bond_Vw_Ret']*df3['w_bond_levered'] + df3['w_rf_levered']*df3['rf30'] - df3['rf30']
df3['Excess_Levered_RP'] = df3['Excess_Levered_RP_Ret'].cumsum()


plt.plot(df3[['Excess_Unlevered_RP','Excess_Levered_RP','Excess_60_40']])
plt.legend(['Excess_Unlevered_RP','Excess_Levered_RP','Excess_60_40'])


\begin{tabular}{rrrrrr}
\toprule
 Excess Return &   T-stat &  Volatility &  Sharpe Ratio &  Skewness &  Excess Kurtosis \\
\midrule
      7.010859 & 3.321589 &   18.937531 &      0.370210 &  0.283482 &         7.924698 \\
      1.555893 & 4.332616 &    3.222015 &      0.482894 & -0.051621 &         4.867099 \\
      7.051232 & 3.368537 &   18.781127 &      0.375442 &  0.298079 &         7.798625 \\
      4.828873 & 3.723417 &   11.635954 &      0.414996 &  0.277261 &         7.847129 \\
      2.256459 & 4.786004 &    4.230117 &      0.533427 &  0.069378 &         4.821429 \\
     10.169402 & 4.877543 &   18.706489 &      0.543630 & -0.350204 &         2.012550 \\
\bottomrule
\end{tabular}



<ipython-input-30-da08f58d148b>:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(outputQ4_2.to_latex(index = False))


In [ ]:


outputQ3 = df3[['Year','Month','Stock_Excess_Vw_Ret','Bond_Excess_Vw_Ret','Excess_Vw_Ret','Excess_60_40_Ret','Stock_inverse_sigma_hat','Bond_inverse_sigma_hat','Unlevered_k','Excess_Unlevered_RP_Ret','levered_k','Excess_Levered_RP_Ret']]

print(outputQ3)
print(outputQ3.dtypes)


Replicate and report Panel A of Table 2 in Asness, Frazzini, and Pedersen (2012), except
for Alpha and t-stat of Alpha columns. Specifically, for all strategies considered, report the
annualized average excess returns, t-statistic of the average excess returns, annualized volatility,
annualized Sharpe Ratio, skewness, and excess kurtosis. Your sample should be from January
1930 to June 2010, at monthly frequency.

In [ ]:


df4 = df3.copy()


df4 = df4.loc[(df4.date_MY >= '1930-01') & (df4.date_MY <= '2010-06') ].copy()

df4 = df4[['Stock_Excess_Vw_Ret','Bond_Excess_Vw_Ret','Excess_Vw_Ret','Excess_60_40_Ret','Excess_Unlevered_RP_Ret','Excess_Levered_RP_Ret']]
# =============================================================================
#
# df4['RP minus value-weighted'] = df4.Excess_Levered_RP_Ret - df4.Excess_Vw_Ret
# df4['RP minus 60/40'] = df4.Excess_Levered_RP_Ret - df4.Excess_60_40_Ret
#
# =============================================================================
df5 = df4.rename(columns = { 'Stock_Excess_Vw_Ret': 'CRSP stocks', 'Bond_Excess_Vw_Ret':'CRSP bonds' ,'Excess_Vw_Ret':'Value-weighted portfolio', 'Excess_60_40_Ret': '60/40 portfolio' ,  'Excess_Unlevered_RP_Ret':'RP, unlevered' , 'Excess_Levered_RP_Ret':  'RP'}).copy()

df6=df5.copy()*12

outputQ4 = df6.apply(lambda x: pd.Series({'Excess Return': x.mean()*100,  'T-stat' : x.mean()/x.std()*np.sqrt(len(df6)) ,'Volatility': x.std()/np.sqrt(12)*100, 'Sharpe Ratio': x.mean()/x.std()*np.sqrt(12) ,'Skewness': x.skew(), 'Excess Kurtosis': x.kurtosis() })).T


outputQ4_2 = df5.apply(lambda x: pd.Series({'Excess Return': x.mean()*100*12,  'T-stat' : x.mean()/x.std()*np.sqrt(len(df5)) ,'Volatility': x.std()*np.sqrt(12)*100, 'Sharpe Ratio': x.mean()/x.std()*np.sqrt(12) ,'Skewness': x.skew(), 'Excess Kurtosis': x.kurtosis() })).T





In [ ]:

print(outputQ4_2.to_latex(index = False))










Index(['PERMNO', 'SHRCD', 'EXCHCD', 'TICKER', 'COMNAM', 'CUSIP', 'DLPDT',
       'DLSTCD', 'DLRETX', 'DLRET', 'PRC', 'RET', 'SHROUT', 'date'],
      dtype='object')

###### Transform the date into Y M D and do MonthEnd

In [ ]:
crsp = pd.read_csv('CRSP_Stocks.csv')
crsp['clean_date'] = pd.to_datetime(crsp['date'],format='%Y-%m-%d',errors='ignore')
crsp['clean_date2'] = crsp['clean_date']+MonthEnd(0)
crsp['clean_date2']

In [ ]:
crsp.drop(['date', 'clean_date'], axis='columns', inplace=True)

In [ ]:
crsp = crsp.rename(columns={'clean_date2': 'date'})
crsp.columns

,PERMNO,SHRCD,EXCHCD,TICKER,COMNAM,CUSIP,DLPDT,DLSTCD,DLRETX,DLRET,PRC,RET,SHROUT,date,ME
0,10026,11.0,3.0,JJSF,J & J SNACK FOODS CORP,46603210,0,0.0,0,0,154.35001,0.067501,18783.0,2019-01-31,2.899156e+06
1,10026,11.0,3.0,JJSF,J & J SNACK FOODS CORP,46603210,0,0.0,0,0,155.28000,0.006025,18783.0,2019-02-28,2.916624e+06
2,10026,11.0,3.0,JJSF,J & J SNACK FOODS CORP,46603210,0,0.0,0,0,158.84000,0.026146,18815.0,2019-03-31,2.988575e+06
3,10026,11.0,3.0,JJSF,J & J SNACK FOODS CORP,46603210,0,0.0,0,0,157.17999,-0.010451,18817.0,2019-04-30,2.957656e+06
4,10026,11.0,3.0,JJSF,J & J SNACK FOODS CORP,46603210,0,0.0,0,0,160.85001,0.023349,18817.0,2019-05-31,3.026715e+06


In [ ]:
crsp.head(2)

In [ ]:
# Keeping common shares (share codes 10 and 11) and major exchanges (exchage code 1, 2 and 3)
crsp = crsp[(crsp['SHRCD'] == 10)| (crsp['SHRCD']  == 11)]
crsp = crsp[(crsp['EXCHCD']== 1) | (crsp['EXCHCD'] == 2) | (crsp['EXCHCD'] == 3)]

In [ ]:
crsp['DLRET'] = crsp['DLRET'].fillna(0)
crsp['RET']   = crsp['RET'].fillna(0)
crsp['DLPDT']   = crsp['DLPDT'].fillna(0)
crsp['DLSTCD']   = crsp['DLSTCD'].fillna(0)
crsp['DLRETX']   = crsp['DLRETX'].fillna(0)

In [ ]:
crsp['ME'] = crsp['PRC'].abs()*crsp['SHROUT']


In [ ]:
crsp['daten'] = crsp['date'].dt.year*12 + crsp['date'].dt.month
crsp['IsValidLag'] = crsp['daten'].diff(1) == 1 # Lag date has to be the lagged date
crsp.loc[crsp[crsp['PERMNO'].diff(1) != 0].index, ['IsValidLag']] = False # Lagged data has to be the same security
crsp['LME'] = crsp[['PERMNO', 'ME']].groupby('PERMNO').shift(1)
crsp.loc[crsp[crsp['IsValidLag'] == False].index, ['LME']] = np.nan
crsp = crsp.drop(['IsValidLag', 'daten'], axis=1)


crsp = crsp.loc[crsp['LME'] > 0, :].copy()

crsp = crsp.sort_values(by = ['date', 'PERMNO']).reset_index().drop('index', axis = 1).copy()

vwret = crsp.groupby(['date'])[['LME','RET']].apply(lambda x: np.sum(x['LME']/x['LME'].sum()*x['RET']))

vwret = vwret.to_frame()
vwret.columns=['vwret']